In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType

In [6]:
path = "student_lifestyle_dataset.csv"

In [7]:
spark = SparkSession.builder.appName("A5").getOrCreate()
df = spark.read.csv(path, header=True, inferSchema=True)
# Columns Available:
#    Student_ID
#    Study_Hours_Per_Day
#    Extracurricular_Hours_Per_Day
#    Sleep_Hours_Per_Day
#    Social_Hours_Per_Day
#    Physical_Activity_Hours_Per_Day
#    Stress_Level
#    Gender
#    Grades
df.select("Student_ID", "Study_Hours_Per_Day", "Extracurricular_Hours_Per_Day", "Sleep_Hours_Per_Day", "Grades").show(5)

+----------+-------------------+-----------------------------+-------------------+------+
|Student_ID|Study_Hours_Per_Day|Extracurricular_Hours_Per_Day|Sleep_Hours_Per_Day|Grades|
+----------+-------------------+-----------------------------+-------------------+------+
|         1|                6.9|                          3.8|                8.7|  7.48|
|         2|                5.3|                          3.5|                8.0|  6.88|
|         3|                5.1|                          3.9|                9.2|  6.68|
|         4|                6.5|                          2.1|                7.2|   7.2|
|         5|                8.1|                          0.6|                6.5|  8.78|
+----------+-------------------+-----------------------------+-------------------+------+
only showing top 5 rows

